In [ ]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)  # Initial hidden state
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

input_size = 768
hidden_size = 128  # You can adjust the hidden size as needed
output_size = 85

model = SimpleRNN(input_size, hidden_size, output_size)

# Example input
x = torch.randn(32, 10, input_size)  # Batch size of 32, sequence length of 10
output = model(x)
print(output.shape)  # Should be [32, 85]

# Define loss function
criterion = nn.CrossEntropyLoss()

# Example target
target = torch.randint(0, output_size, (32,))

# Compute loss
loss = criterion(output, target)
print(loss.item())

In [1]:
def txt_to_sequences(txt_file):
    sequences = []
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Extract the sequence
            sequence = line.split('[')[1].split(']')[0]
            sequence = sequence.split('),')
            sequence = [tuple(map(int, x.replace('(', '').replace(')', '').split(','))) for x in sequence]
            sequences.append(sequence)

    return sequences

In [ ]:
import pandas as pd
txt_file = r"..\results_PartB\output_hiroshima\frequent_sequences_filtered_20241112_135806.txt"
dataset = txt_to_sequences(txt_file)
dataset

def sequence_to_vectors(tup_sequence, path_to_mapping_csv):
    
    vector_sequence = []
    # Load the mapping
    df = pd.read_csv(path_to_mapping_csv)
    df["POI_Probabilities"] = df.iloc[:,2:(2+85)].apply(lambda x: torch.tensor(x.values), axis=1)
    df['Input_features'] = df.iloc[:,(2+85):(2+85+768)].apply(lambda x: torch.tensor(x.values), axis=1)
    
    for tup in tup_sequence:
        x, y = tup
        
    
    

[[(80, 96), (80, 95)],
 [(79, 94), (79, 94)],
 [(80, 95), (80, 96)],
 [(80, 96), (80, 96)],
 [(80, 93), (80, 92)],
 [(80, 92), (80, 93)],
 [(79, 93), (79, 92)],
 [(80, 93), (80, 93)],
 [(45, 109), (45, 109)],
 [(80, 92), (80, 92)],
 [(79, 92), (79, 93)],
 [(79, 92), (80, 92)],
 [(73, 94), (74, 94)],
 [(80, 95), (80, 95)],
 [(74, 94), (74, 94)],
 [(80, 92), (79, 92)],
 [(74, 94), (73, 94)],
 [(78, 94), (79, 94)],
 [(79, 94), (79, 94), (79, 94)],
 [(79, 92), (79, 92)],
 [(79, 93), (79, 93)],
 [(80, 93), (79, 93)],
 [(73, 111), (74, 111)],
 [(79, 85), (80, 85)],
 [(74, 111), (74, 111)],
 [(74, 111), (73, 111)],
 [(73, 94), (73, 94)],
 [(79, 94), (78, 94)],
 [(79, 93), (79, 94)],
 [(134, 142), (133, 141)],
 [(80, 85), (79, 85)],
 [(79, 85), (79, 85)],
 [(77, 87), (76, 87)],
 [(79, 94), (79, 94), (79, 94), (79, 94)],
 [(133, 141), (134, 142)],
 [(134, 142), (134, 142)],
 [(95, 87), (95, 88)],
 [(79, 94), (79, 93)],
 [(133, 141), (133, 141)],
 [(50, 112), (50, 112)],
 [(174, 7), (174, 7)],
 

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SequenceDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        input_sequence = self.sequences[idx][:-1]
        target_label = self.sequences[idx][-1]
        return 